input :

[
    {
        "type": "text",
        "text": "UndergraduateProspectus",
        "text_level": 1,
        "page_idx": 0
    },
    {
        "type": "image",
        "img_path": "images/ca5dfe4537f20eab2806e7f379389553f48f18730bf480f8f4ffc43df30a11ec.jpg",
        "image_caption": [],
        "image_footnote": [],
        "page_idx": 1
    },
    {
        "type": "text",
        "text": "CONTENTS",
        "text_level": 1,
        "page_idx": 1
    },
    {
        "type": "text",
        "text": "2 TOP 10 REASONS WHY NTU SMART CAMPUS IS FOR YOU 4 BE AMONG THE TOP RANKED 6 NURTURING TALENT FOR THE GOOD 8 A TRANSFORMATIVE LEARNING",
        "page_idx": 1
    },
    {
        "type": "text",
        "text": "10 PREMIER SCHOLARS PROGRAMMES",
        "text_level": 1,
        "page_idx": 1
    },
    {
        "type": "table",
        "img_path": "images/f0f54084622cac60b63859683860e54002612b7c339227eeca17182f824a7ea0.jpg",
        "table_caption": [
            "APPLICATION PERIOD AND FEE"
        ],
        "table_footnote": [],
        "table_body": "<table><tr> ... </tr></table>",
        "page_idx": 24
    },
    {
        "type": "equation",
        "text": "\n$$\nI_{l} = \\left|\\sum_{i}A_{h,i}\\odot \\frac{\\partial\\mathcal{L}(x)}{\\partial A_{h,i}}\\right|. \\tag{1}\n$$\n",
        "text_format": "latex",
        "page_idx": 1
    },
    ...
]

enhance and output:
[
    {
        "type": "text",
        "text": "UndergraduateProspectus",
        "node_level": 1, # [dev] text_level change name to node_level
        "page_idx": 0,
        "node_idx": 0 # [dev] add node_idx from i to i++
    },
    {
        "type": "image",
        "img_path": "images/....jpg",
        "image_caption": [],
        "image_footnote": [],
        "text": # OCR the image extract text
        "outline": [] # add from layout.json
        "description": # qwen2.5vl
        "node_level": -1 add node_level, can be -1, to be add a true num
        "page_idx": 1
        "node_idx": 0 # [dev] add node_idx from i to i++
    },
    ...
    {
        "type": "table",
        "img_path": "images/....jpg",
        "table_caption": [
            "APPLICATION PERIOD AND FEE"
        ],
        "table_footnote": [],
        "table_body": "<table><tr> ... </tr></table>",
        "node_level": -1 # add node_level, can be -1, to be add a true num
        "page_idx": 24,
        "outline": [] # add from layout.json
        "node_idx": 0 # [dev] add node_idx from i to i++
    },
    {
        "type": "equation",
        "text": "\n$$\nI_{l} = \\left|\\sum_{i}A_{h,i}\\odot \\frac{\\partial\\mathcal{L}(x)}{\\partial A_{h,i}}\\right|. \\tag{1}\n$$\n",
        "text_format": "latex",
        "page_idx": 1,
        "node_level": -1 add node_level, can be -1, to be add a true num
        "node_idx": 0
    },
]


In [9]:
from utils import load_json
from typing import List
import fitz  # pymupdf
from IPython.display import Image, display
from io import BytesIO
def save_cropped_image(pdf_path: str, page_num: int, outline: List[float]) -> str:
    """
    保存裁剪后的图像到指定目录
    
    Args:
        pdf_path: PDF文件路径
        page_num: 页码（从0开始）
        outline: 坐标 [x1, y1, x2, y2]
        doc_name: 文档名称（用于创建子目录）
        element_index: 元素索引，用于文件命名
        base_output_dir: 基础输出目录（dom_output_path）
        
    Returns:
        保存的图像文件相对路径
    """
    with pymupdf.open(pdf_path) as pdf:
        if page_num >= len(pdf):
            raise ValueError(f"Page {page_num} does not exist in PDF")
        
        page = pdf[page_num]
        # 创建裁剪矩形
        rect = pymupdf.Rect(outline[0], outline[1], outline[2], outline[3])
        
        # 获取裁剪区域像素图
        pix = page.get_pixmap(clip=rect, dpi=150)
    
    return pix
enhanced_content_list = load_json("./../content_list.adapted.json")

for i in enhanced_content_list:
    if i['type'] == 'image':
        print(i['page_idx'])
        pix = save_cropped_image('/data/users/yiming/dtagent/MMLongBench/documents/welcome-to-nus.pdf', i['page_idx'],i['outline'])
        # 保存为 PNG 到内存
        img_bytes = BytesIO(pix.tobytes("png"))

        # 显示图像
        # display(Image(data=img_bytes.getvalue()))

0
1
2
2
2
2
2
2
3
3
3
3
4
4
4
5
5
5


6
6
6
6
6
6
7
7
8
8
9
9
9
10
11
11
12
12
12
13
13
14
14
14
14
14
15
15
15
16
16
17
17
18
18
19
19
20
21
21
22
22
23
23


目标与约束

  - 输入：单个文档的 doctree.json（flat），以及每页的“页级计划”plan（来自 Qwen/GPT，离线提供）。
  - 输出：新的 flat doctree（例如写为 doctree.vl.json），字段结构与原 flat 一致：
      - 仅修改页面内节点顺序和文本节点的 node_level。
      - 合并相邻标题：只通过“修改 into 节点 text、删除 from 节点”的方式体现。
      - 补充被忽略标题：新增标准 text 节点 {type, text, page_idx, node_level}。
      - 重新分配 node_idx（0..n-1）并重建 indices（by_page、by_type、id_to_idx）。
      - 不添加任何新的自定义键。

  页级计划 plan（供模型输出/校验）

  - 必填：
      - page_idx: int
      - items: [{ node_id: str, level: int }]：必须是本页原有 node_id 的一个排列（无遗漏/重复/越权），level 在 [1..4]
  （可调）。
  - 可选：
      - merges: [{ into: node_id, from: [node_id] }]：仅允许 text 类型、同页、在 items 重排后相邻；规模小（每条 from
  数量≤2-3）。
      - virtual_titles: [{ text: str, level: int, insert_after?: node_id }]：新增标题（仅标准字段）；插入到本页指定位
  置（或页首/页末）。
  - 严格禁止：在 plan 中出现除上述字段以外的键；在 items/merges 中出现文本域。

  应用算法（对单页）

  - 收集本页节点 ID 集合 allowed_ids。
  - 校验 plan：
      - items 全覆盖且仅覆盖 allowed_ids；level 合法。
      - merges 的 into/from 均在 items；类型均为 text；应用 items 重排后验证相邻性。
      - virtual_titles 文本长度适中（如≤80），level 合法；insert_after 若存在必须在本页 items 中。
  - 应用顺序与定级：
      - 按 items 重排本页的 children 子序列。
      - 将 level 写入对应节点的 node_level。
  - 应用合并：
      - 对每条 merge，按重排后的相邻顺序执行：into.text = concat(into.text, from.text)；删除 from 节点。
      - into 的 node_level 以 plan.items 中 into 的 level 为准。
  - 插入虚拟标题：
      - 新建标准 text 节点：{type: "text", text, page_idx, node_level: level}。
      - 插入到指定 insert_after 的后面；若未指定，默认本页子序列最前（或最后）。
  - 该页处理结果替换回全局 children 列表；其他页保持不变。

  收尾（全文级）

  - 重新线性赋值 node_idx（0..n-1）。
  - 重建 indices：
      - by_page: { page_idx -> [node_idx,...] }（内部用 int 键，JSON 序列化会转为字符串表现）
      - by_type: { type -> [node_idx,...] }
      - id_to_idx: { "{doc_id}#{node_idx}" -> node_idx }
  - 写出新的 flat doctree（例如 doctree.vl.json）；不覆盖原 doctree.json。

  CLI 计划（先实现“应用”与“校验”，模型调用后接入）

  - python -m tree.page_agent_cli --in-file /dir/doctree.json --pages 3,4 --plan-dir /dir/plans --out /dir/
  doctree.vl.json
      - --pages: 逗号与区间混合（如 1,2,5-7）；或 --pages all
      - --plan-dir: 每页一个 page_{page_idx}.json，内容为上面的 plan
      - --no-indices: 如需跳过 indices 重建（默认重建）
  - 可选单页调试：--plan-file /dir/page_3.json